In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import math
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler


In [6]:
def get_target(df, target_name):
    return df.drop(target_name, axis=1), df[target_name]

def drop_correlated_features(df, threshold):
    corr_matrix = df.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    df = df.drop(to_drop, axis=1)
    return df

def drop_min_rows(df, *values):
    for value in values:
        df = df.drop(df[df['DP'] == value].index, axis=0)
    return df

def balance_classes(df):
    df_balanced = pd.DataFrame()
    min_class_count = df['DP'].value_counts().min()
    for label in df['DP'].unique():
        df_label = df[df['DP'] == label]
        if len(df_label) > min_class_count:
            df_random_sample = df_label.sample(min_class_count)
            df_balanced = pd.concat([df_balanced, df_random_sample])
        else:
            df_balanced = pd.concat([df_balanced, df_label])
    return df_balanced

In [7]:
from posixpath import split
# load dataset

df = pd.read_csv("Data.csv", header = 0, delimiter = ";")
print("drop duplicates")
df = df.drop_duplicates()
constant_features = [column for column in df.columns if df[column].nunique() == 1]
print("drop constant features = ",len(constant_features))
df.drop(columns=constant_features, inplace=True)
print("split features and target")

df_features, df_target = get_target(df, 'DP')

df_corr = drop_correlated_features(df_features, 0.9)

df = pd.concat([df_corr, df_target], axis=1)
new_df = drop_min_rows(df, -60, -140)

#new_df = balance_classes(new_df)

df_check = new_df.copy()

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif, f_classif, chi2

kbest = SelectKBest(k=200, score_func=mutual_info_classif)
kbest.fit(df_check.drop('DP', axis=1), df_check['DP'])
rec = kbest.get_support()
print(rec)
list_best = []
for i, r in enumerate(rec):
    if r == True:
        list_best.append(df_check.columns[i])
        print(df_check.columns[i], kbest.scores_[i])
if 'DP' in list_best:
    list_best.remove('DP')
df_best = df_check[list_best]

# check that dataframe contain column

if df_best.Disbalance.empty == None:
    df_best =  pd.concat([df_check.Disbalance, df_best], axis=1)

df_best.head()

drop duplicates
drop constant features =  1361
split features and target
[ True  True False  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True False  True  True  True
 False  True  True  True  True  True  True  True  True False  True  True
  True  True  True False  True  True  True  True  True  True  True  True
  True  True  True  True False  True False  True  True  True  True  True
  True False False  True  True False  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True False  True False  True  True  True  True  True False  True  True
 False  True False  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  T

,Disbalance,ON_LINE gen 2,ON_LINE gen 3,PG gen 3,QG gen 3,PG gen 5,PG gen 11,QG gen 11,PG gen 12,QG gen 26,...,Q_BEG line 206,Q_END line 206,Q_BEG line 208,P_BEG line 220,I_BEG line 220,Q_BEG line 300,Q_BEG line 310,I_END line 312,P_BEG line 317,I_END line 317
5,143.14,0,0,0.0,0.0,0.0,68.05,6.88,0.0,0.0,...,-0.47,0.5,0.13,0.0,0.0,-0.39,3.38,0.0,0.81,3.98
15,142.77,0,0,0.0,0.0,0.0,67.66,7.02,0.0,0.0,...,-0.47,0.5,0.13,0.0,0.0,-0.38,3.34,0.0,0.49,2.44
16,142.60,0,0,0.0,0.0,0.0,68.17,7.14,0.0,0.0,...,-0.47,0.5,0.13,0.0,0.0,-0.40,3.39,0.0,0.00,0.28
18,143.20,0,0,0.0,0.0,0.0,68.17,7.07,0.0,0.0,...,-0.47,0.5,0.14,0.0,0.0,-0.38,3.38,0.0,0.00,0.28
19,142.49,0,0,0.0,0.0,0.0,67.78,7.12,0.0,0.0,...,-0.47,0.5,0.13,0.0,0.0,-0.39,3.38,0.0,0.00,0.28


In [32]:
# Boruta feature selection

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from boruta import BorutaPy

np.int = np.int32
np.float = np.float64
np.bool = np.bool_

x = df_check.drop('DP', axis=1)
y = df_check['DP']


print(x.values, y.values)

# convert to numpy
#features = 


feature_selector = BorutaPy(RandomForestClassifier(n_jobs = -1, max_depth = 5),
)
feature_selector.fit(x.values, y.values)
rec = feature_selector.support_
best_boruto = []

for i, r in enumerate(rec):
    if r == True:
        best_boruto.append(df_check.columns[i])

print(len(best_boruto))
# make dataframe from columns list

df_best_boruto = df_check[best_boruto]        

[[1.4314e+02 0.0000e+00 0.0000e+00 ... 0.0000e+00 8.1000e-01 3.9800e+00]
 [1.4277e+02 0.0000e+00 0.0000e+00 ... 0.0000e+00 4.9000e-01 2.4400e+00]
 [1.4260e+02 0.0000e+00 0.0000e+00 ... 0.0000e+00 0.0000e+00 2.8000e-01]
 ...
 [8.7250e+01 1.0000e+00 0.0000e+00 ... 9.8000e-01 1.0000e-02 1.3600e+00]
 [8.6290e+01 1.0000e+00 0.0000e+00 ... 9.3000e-01 0.0000e+00 1.3500e+00]
 [8.6750e+01 1.0000e+00 0.0000e+00 ... 9.3000e-01 0.0000e+00 1.3500e+00]] [-130 -130 -130 ...  -90  -90  -90]
Disbalance
ON_LINE gen 2
PG gen 3
QG gen 3
PG gen 5
PG gen 11
QG gen 11
PG gen 12
QG gen 26
PG gen 27
QG gen 27
U node 7001
FAZA node 7001
PN node 7001
QN node 7001
U node 7003
FAZA node 7003
QN node 7005
PN node 7008
QN node 7009
PN node 7010
QN node 7010
QN node 7011
U node 7012
FAZA node 7012
U node 7015
FAZA node 7015
PN node 7015
QN node 7015
PN node 7016
QN node 7019
QN node 7023
QN node 7025
FAZA node 7027
QN node 7029
PN node 7030
QN node 7030
U node 7032
QN node 7032
QN node 7033
PN node 7035
QN node 7038


In [41]:
# pytorch multiclass classification

from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

data_labels = df_check["DP"]
data_labels = pd.factorize(data_labels)[0]
data_features = df_best_boruto

# pytorch model

class Dataset(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
    def __len__ (self):
        return len(self.X_data)

train_features, test_features, train_labels, test_labels = train_test_split(data_features, data_labels, test_size = 0.33, random_state = 42)

train_features = torch.from_numpy(train_features.values).float()

test_features = torch.from_numpy(test_features.values).float()

train_labels = torch.from_numpy(train_labels).long()

test_labels = torch.from_numpy(test_labels).long()

train = Dataset(train_features, train_labels)

test = Dataset(test_features, test_labels)

train_loader = DataLoader(dataset = train, batch_size = 64, shuffle = True)

test_loader = DataLoader(dataset = test, batch_size = 64, shuffle = True)

class MLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 64)
        self.fc5 = nn.Linear(64, 64)
        self.fc6 = nn.Linear(64, 64)
        self.fc7 = nn.Linear(64, output_dim)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = F.relu(self.fc6(x))
        x = self.fc7(x)
        return x

model = MLP(data_features.shape[1], 6)

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adamax(model.parameters(), lr=0.001)

for epoch in range(1000):
    for i, (features, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step(closure=None)
model.eval()

with torch.no_grad():
    correct = 0
    total = 0
    for test_features, test_labels in test_loader:
        outputs = model(test_features)
        _, predicted = torch.max(outputs.data, 1)
        total += test_labels.size(0)
        correct += (predicted == test_labels).sum().item()

    print('Accuracy of the model on the test features: {} %'.format(100 * correct / total))



print(model)

Accuracy of the model on the test features: 95.52147239263803 %
MLP(
  (fc1): Linear(in_features=186, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=64, bias=True)
  (fc5): Linear(in_features=64, out_features=64, bias=True)
  (fc6): Linear(in_features=64, out_features=64, bias=True)
  (fc7): Linear(in_features=64, out_features=6, bias=True)
)
